In [1]:
import numpy 
import pandas

import nest_asyncio
import asyncio

import bleak
from bleak import BleakScanner, BleakClient

nest_asyncio.apply()

In [2]:
def adv_decrypt(data):
    # Расшифровка уровня топлива
    oil_level_raw = int.from_bytes(data[1:3], byteorder='little', signed=False)
    print(f"Уровень топлива: {oil_level_raw}")

    # Расшифровка напряжения встроенной батареи
    battery_voltage_raw = int.from_bytes(data[3:4], byteorder='little', signed=False)
    battery_voltage = battery_voltage_raw / 10.0
    print(f"Напряжение встроенной батареи: {battery_voltage} В")

    # Расшифровка температуры
    TD_temp_raw = int.from_bytes(data[4:5], byteorder='little', signed=True)
    print(f"Температура: {TD_temp_raw} Градус Цельсия")

    # Расшифровка версии прошивки
    TD_version_raw = int.from_bytes(data[5:6], byteorder='little', signed=False)
    print(f"Версия прошивки: {TD_version_raw}")
                    
    # Расшифровка периода (непосредственно измеренный параметр)
    TD_period_param = int.from_bytes(data[6:10], byteorder='little', signed=False)
    print(f"Период: {TD_period_param} ")

    # Расшифровка режима работы
    TD_mode_raw = int.from_bytes(data[10:11], byteorder='little', signed=False)
    TD_range = (TD_mode_raw & 0b11110000)
    if TD_range & 0x80:
        print("Диапазон датчика 4095")
    if TD_range & 0b00000000:
         print("Диапазон датчика 1024")

    TD_flags = (TD_mode_raw & 0b00001111)
    if TD_flags & 0b00000000:
        print("Обычный алгоритм расчета")
    if TD_flags & 0b00000001:
        print("Алгоритм расчета с тарировочной таблицей")

      # Расшифровка сглаживания и термокомпенсации
    TD_smoth_and_copm_raw = int.from_bytes(data[11:12], byteorder='little', signed=False)
    TD_therm_compensation = (TD_smoth_and_copm_raw & 0b11110000)
    if TD_therm_compensation & 0x80:
         print("Термокомпенсация выключена")
    if TD_therm_compensation & 0b00000000:
          print("Термокомпенсация включена")

    TD_smoothing = TD_smoth_and_copm_raw & 0x0F
    print(f'Сглаживание: {TD_smoothing}')

In [3]:
timeout_seconds = 60

In [4]:
NAME = 'TD_383449'

In [5]:
import struct
import datetime



class MyScanner:
    def __init__(self):
        self._scanner = BleakScanner()
        self._scanner.register_detection_callback(self.detection_callback)
        self.scanning = asyncio.Event()

    def detection_callback(self, device, advertisement_data):
        if device.name is not None:
            if device.name == NAME: # device.name.startswith('TD_'):
                print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, device.rssi), device.metadata['uuids'], datetime.datetime.now())
                adv_decrypt(advertisement_data.manufacturer_data[3862])
    async def run(self):
        await self._scanner.start()
        self.scanning.set()
        end_time = loop.time() + timeout_seconds
        while self.scanning.is_set():
            if loop.time() > end_time:
                self.scanning.clear()
                print('\t\tScan has timed out so we terminate')
            await asyncio.sleep(0.1)
        await self._scanner.stop()


if __name__ == '__main__':
    my_scanner = MyScanner()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(my_scanner.run())

C:\Users\Dukalis\AppData\Local\Temp\ipykernel_8944\1427519126.py:9: FutureWarning: This method will be removed in a future version of Bleak. Use the detection_callback of the BleakScanner constructor instead.
  self._scanner.register_detection_callback(self.detection_callback)
C:\Users\Dukalis\AppData\Local\Temp\ipykernel_8944\1427519126.py:15: FutureWarning: BLEDevice.rssi is deprecated and will be removed in a future version of Bleak, use AdvertisementData.rssi instead
  print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, device.rssi), device.metadata['uuids'], datetime.datetime.now())
C:\Users\Dukalis\AppData\Local\Temp\ipykernel_8944\1427519126.py:15: FutureWarning: BLEDevice.metadata is deprecated and will be removed in a future version of Bleak, use AdvertisementData instead
  print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, device.rssi), device.metadata['uuids'], datetime.datetime.now())


Device E2:F1:F3:8C:E1:45 (TD_383449), RSSI=-40 dB [] 2023-05-26 15:02:12.169303
Уровень топлива: 1
Напряжение встроенной батареи: 3.7 В
Температура: 23 Градус Цельсия
Версия прошивки: 141
Период: 30511 
Термокомпенсация выключена
Сглаживание: 0
Device E2:F1:F3:8C:E1:45 (TD_383449), RSSI=-36 dB [] 2023-05-26 15:03:00.224145
Уровень топлива: 1
Напряжение встроенной батареи: 3.7 В
Температура: 23 Градус Цельсия
Версия прошивки: 141
Период: 30511 
Термокомпенсация выключена
Сглаживание: 0
		Scan has timed out so we terminate


In [6]:
ADDRESS = 'E2:F1:F3:8C:E1:45'

In [7]:
SERVICE_UUID = '6e400001-b5a3-f393-e0a9-ffeeddc5d9d9'

In [8]:
uniq_uuid_part = SERVICE_UUID.split('-')[-1]

In [9]:
RX_UUID_part = '6e400003-b5a3-f393-e0a9-'

In [10]:
TX_UUID_part = '6e400002-b5a3-f393-e0a9-'

In [11]:
RX_UUID = RX_UUID_part + uniq_uuid_part

In [12]:
RX_UUID

'6e400003-b5a3-f393-e0a9-ffeeddc5d9d9'

In [13]:
TX_UUID = TX_UUID_part + uniq_uuid_part

In [14]:
TX_UUID

'6e400002-b5a3-f393-e0a9-ffeeddc5d9d9'

In [21]:
import asyncio
from bleak import BleakClient

MESSAGE = b"HK:1:100\r"

async def client():
    async with BleakClient(ADDRESS, timeout=40, is_secured=True) as client:
        # Connect to the device and discover the services and characteristics
        services = await client.get_services()


        # Find the RX and TX characteristics
        rx_char, tx_char = None, None  # обнуляем переменные
        for service in services:
            if service.uuid == SERVICE_UUID:
                for characteristic in service.characteristics:

                    if characteristic.uuid == RX_UUID:
                        rx_char = characteristic

                    elif characteristic.uuid == TX_UUID:
                        tx_char = characteristic
                        
        # Subscribe to notifications for the RX characteristic
        await client.start_notify(rx_char.handle, callback)
                        
        # Записываем сообщение в TX характеристику 
        await client.write_gatt_char(tx_char.handle, MESSAGE)
        print(f'WTIRE: {MESSAGE} on TX_char')

       # # Wait for notifications
        #await asyncio.sleep(10)

        # Stop receiving notifications
        #await client.stop_notify(rx_char.handle)

async def callback(sender: int, data: bytearray):
    print(f'Received notification from RX_char: {data}')

loop = asyncio.get_event_loop()
loop.run_until_complete(client())

BleakError: Could not get GATT services: Unreachable

In [53]:
'''     if device.address == address_to_look_for:
            byte_data = advertisement_data.service_data.get(service_id_to_look_for)
            num_to_test, = struct.unpack_from('<I', byte_data, 0)
            if num_to_test == 62976:
                print('\t\tDevice found so we terminate')
                self.scanning.clear()
                
'''

"     if device.address == address_to_look_for:\n            byte_data = advertisement_data.service_data.get(service_id_to_look_for)\n            num_to_test, = struct.unpack_from('<I', byte_data, 0)\n            if num_to_test == 62976:\n                print('\t\tDevice found so we terminate')\n                self.scanning.clear()\n                \n"